In [58]:
import os

In [59]:
%pwd

'C:\\ML_OPS\\Projects\\project1'

In [60]:
os.chdir("C:\ML_OPS\Projects\project1")
# os.chdir("project1")
%pwd

%pwd

'C:\\ML_OPS\\Projects\\project1'

In [61]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [62]:
import pandas as pd 
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [64]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [65]:
data.shape

(1599, 12)

In [66]:
from dataclasses import dataclass 
from pathlib import Path
@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [67]:
from src.DataScienceProject.utils.shared import read_yaml, create_directories
from src.DataScienceProject.constants import *

In [68]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir= config.unzip_data_dir,
            all_schema=schema
        )    
        return data_validation_config


In [69]:
import os
from src.DataScienceProject import logger


In [70]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = False
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    logger.info(f'Column {col} is not in the schema.')
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation Status is : {validation_status}')
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation Status is : {validation_status}')
            return validation_status

        except Exception as e:
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f'Validation Status is : {e}')
            logger.error(f'An error occurred while validating the data: {e}')
            raise e

    def validate_column_types(self) -> bool:
        """
        Validate that each column has the expected data type according to the schema.
        """
        try:
            data = pd.read_csv(self.config.unzip_data_dir)
            schema = self.config.all_schema
            status = True

            for column, expected_type in schema.items():
                if column in data.columns:
                    actual_dtype = str(data[column].dtype)
                    # Simplified dtype comparison
                    if expected_type == "int" and not pd.api.types.is_integer_dtype(data[column]):
                        logger.warning(f"Column '{column}' is expected to be int but found {actual_dtype}")
                        status = False
                    elif expected_type == "float" and not pd.api.types.is_float_dtype(data[column]):
                        logger.warning(f"Column '{column}' is expected to be float but found {actual_dtype}")
                        status = False
                    elif expected_type == "object" and not pd.api.types.is_object_dtype(data[column]):
                        logger.warning(f"Column '{column}' is expected to be object/string but found {actual_dtype}")
                        status = False
                else:
                    logger.warning(f"Column '{column}' not found in dataset")
                    status = False

            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f'\nColumn Type Validation Status: {status}')

            return status

        except Exception as e:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f'\nColumn Type Validation Error: {e}')
            logger.error(f'Error during column type validation: {e}')
            raise e


In [71]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_column_types()
except Exception as e:
    raise e

[2025-06-05 17:09:58,393 : INFO : shared : Yaml File: config\config.yaml loaded successfully!]
[2025-06-05 17:09:58,395 : INFO : shared : Yaml File: params.yaml loaded successfully!]
[2025-06-05 17:09:58,400 : INFO : shared : Yaml File: schema.yaml loaded successfully!]
[2025-06-05 17:09:58,402 : INFO : shared : created directory at : artifacts]
[2025-06-05 17:09:58,405 : INFO : shared : created directory at : artifacts/data_validation]


In [72]:
print(data.dtypes)

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object
